In [291]:
import os 
import pandas as pd

def get_min_max(path):
    df_list = []
    for dir in os.listdir(path):
        for subdir in os.listdir(os.path.join(path,dir)):
            load_path = os.path.join(path, dir, subdir)
            df = pd.read_csv(load_path)
            df_list.append(df)
    df = pd.concat((df_list[0], df_list[1], df_list[2], df_list[3], df_list[4], df_list[5]))
    return df
    


def create_plateau(n, df):
    plateau_counts = []
    # max_val = float(df.max())
    # print("Max: ",max_val)
    # min_val = float(df.min())
    # print("Min: ",min_val)
    max_val = 1157.6
    min_val = 43.2
    plateau_list = []
    plateau_size = (max_val-min_val)/n
    print("Plateua Size: ",plateau_size)
    current_plateau = min_val
    for i in range(0,n):
        new_plateau = current_plateau + plateau_size
        print(str(current_plateau) + " --- " + str(new_plateau))
        df_filter = int(df[(df["BUS"] > current_plateau) & (df["BUS"] <= new_plateau)].count()[0])
        current_plateau = new_plateau
        print(df_filter)
        plateau_counts.append(df_filter)
    return plateau_counts

def only_max(df, tolerance):
    max_val = float(df.max())
    plateau_count = df[df["BUS"] >= (1157.6-tolerance)].count()[0]
    return plateau_count


def load_data(path, plateau_size=None):
    no_dash_list = []
    dash_list = []

    for dir in os.listdir(path):
        for subdir in os.listdir(os.path.join(path,dir)):
            load_path = os.path.join(path, dir, subdir)
            df = pd.read_csv(load_path)
            df = df[["BUS"]]
            if plateau_size is None:
                plateau_counts = only_max(df,tolerance=10)
            else:
                plateau_counts = create_plateau(plateau_size, df)
            if "no" in subdir:
                no_dash_list.append(plateau_counts)
            else:
                dash_list.append(plateau_counts)
    NoDash = pd.DataFrame(no_dash_list)
    NoDash["Dash"] = "no"
    Dash = pd.DataFrame(dash_list)
    Dash["Dash"] = "yes"
    df = pd.concat((NoDash,Dash))
    return df.sample(frac=1).reset_index(drop=True)
    

In [292]:
df = get_min_max("Printer3D")
df.dropna(inplace=True)
print(df.BUS.max())
print(df.BUS.min())

1157.6
43.2


In [293]:
# if platue_size is not assigned like: load_data(path = "Printer3D")
# then there are no plateaus. Instead there is a tolerance of the maximum value of the current
# and everything above this tolerance is counted 
# Example: max value of current: 1000mA
# Tolerance: 100mA
# Result: Every current above 900mA is counted

# df = load_data(path = "Printer3D")

# with plateaus: assign desired number of plateaus
df = load_data(path = "Printer3D")
df

,0,Dash
0,4,no
1,28,no
2,0,yes
3,2,yes
4,2,yes
5,1,no


In [287]:
# store dataset for train/test model
df.to_csv("3d_printer_plateau.csv", index=False)

In [289]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import cross_val_score 
y = df[["Dash"]]
X = df.drop("Dash", axis=1)
dt = DecisionTreeClassifier()
# don't want to extract testset, instead using Cross Validation
cv_results = cross_val_score(dt, X, y, cv=3)
print(cv_results)

[0.5 1.  0.5]


Clean data_dash0 because of late start

In [256]:
df_1 = pd.read_csv("Printer3D/Dash/data_dash1.csv")
# compare size with another dataset
print(df_1.shape[0])
df = pd.read_csv("data_dash0.csv")
# delete the starting procedure
df = df.loc[40000:]
# store it without index
df.to_csv("Printer3D/Dash/data_dash0_gereinigt.csv", index=False)
# validate size of new datas
df


191235


,Time,MCU,BUS
40000,120324.0,5.5,1022.9
40001,120327.0,5.5,1029.9
40002,120330.0,5.6,1032.6
40003,120333.0,6.0,1033.1
40004,120336.0,5.2,1028.5
...,...,...,...
233155,695864.0,5.8,115.4
233156,695867.0,5.8,117.0
233157,695870.0,6.3,115.8
233158,695873.0,5.8,116.2
